In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

from keras_tuner import HyperModel
from keras_tuner.tuners import RandomSearch, Hyperband, BayesianOptimization

import random
tf.random.set_seed(3407)
np.random.seed(3407)
random.seed(3407)

from helper_functions import (
    get_file_size, 
    convert_bytes, 
    convert_prefetchdataset_to_numpy_arrays,
    predict_and_print_full_results,
    evaluate_time_of_prediction
    )

2024-07-28 17:38:01.240113: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-28 17:38:01.560935: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-28 17:38:01.560991: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-28 17:38:01.616217: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-28 17:38:01.730447: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-28 17:38:01.732306: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [6]:
train_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/training", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
test_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/testing", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
val_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/validation", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)

label_names = np.array(train_dataset.class_names)
print("Classes: ", label_names)

Found 11292 files belonging to 2 classes.


2024-07-28 17:39:04.745030: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
2024-07-28 17:39:04.816988: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available


Found 1393 files belonging to 2 classes.
Found 1380 files belonging to 2 classes.
Classes:  ['non_target' 'target']


In [7]:
x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(train_dataset, data_type="time-series")
x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(val_dataset, data_type="time-series")
x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(test_dataset, data_type="time-series")

### HPO

In [30]:
from tensorflow import keras

In [49]:
class TransformerHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes, num_blocks):
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.num_blocks = num_blocks

    def transformer_encoder(self, inputs, head_size, num_heads, filters_conv, kernel_size, dropout=0):
        # Normalization and Attention
        x = layers.LayerNormalization(epsilon=1e-6)(inputs)
        x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
        x = layers.Dropout(dropout)(x)
        res = x + inputs

        # Feed Forward Part
        x = layers.LayerNormalization(epsilon=1e-6)(res)
        x = layers.Conv1D(filters=filters_conv, kernel_size=kernel_size, activation="relu")(x)
        x = layers.Dropout(dropout)(x)
        x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
        return x + res

    def build(self, hp, dropout=0):
        inputs = layers.Input(shape=self.input_shape)
        
        # Parameters
        filters_conv = hp.Int('filters', min_value=2, max_value=12, step=2)
        # kernel_size = hp.Choice('kernel_size', values=[1, 3])
        pool_size = hp.Int('pool_size', min_value=6, max_value=10, step=2)
        num_heads = hp.Choice('num_heads', values=[1, 2, 3])
        learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')

        x = layers.Conv1D(filters=filters_conv, kernel_size=1, activation="relu")(inputs)
        x = layers.MaxPooling1D(pool_size=pool_size)(x)
        x = layers.AveragePooling1D(pool_size=pool_size)(x)
        for i in range(self.num_blocks):
            head_size = hp.Int(f'head_size_{i}', min_value=4, max_value=16, step=4)
            x = self.transformer_encoder(x, head_size, num_heads, filters_conv, 1, dropout)

        x = layers.GlobalAveragePooling1D()(x)
        x = layers.Dropout(dropout)(x)
        outputs = layers.Dense(self.num_classes, activation="softmax")(x)
        model = keras.Model(inputs, outputs)

        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy']
        )
        return model

In [52]:
input_shape = (48000,1)
num_classes = 2
num_blocks = 1

num_of_conv_layers = 1
tuner_1_block_transformer = RandomSearch(
    TransformerHyperModel(input_shape, num_classes, num_blocks),
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='../hpo_tuner/tiny_transformer',
    project_name='tiny_transformer_time_series_1_block_tuning'
)
tuner_1_block_transformer.search(train_dataset, epochs=3, validation_data=val_dataset)
tuner_1_block_transformer.results_summary()

Trial 5 Complete [00h 05m 14s]
val_accuracy: 0.8057971000671387

Best val_accuracy So Far: 0.8405796885490417
Total elapsed time: 00h 59m 03s
Results summary
Results in ../hpo_tuner/tiny_transformer/tiny_transformer_time_series_1_block_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
filters: 12
pool_size: 6
num_heads: 2
learning_rate: 0.000767170096923145
head_size_0: 12
Score: 0.8405796885490417

Trial 4 summary
Hyperparameters:
filters: 6
pool_size: 8
num_heads: 1
learning_rate: 0.000130573118342512
head_size_0: 4
Score: 0.8057971000671387

Trial 2 summary
Hyperparameters:
filters: 6
pool_size: 10
num_heads: 1
learning_rate: 0.0026489221357360827
head_size_0: 12
Score: 0.7876811623573303

Trial 1 summary
Hyperparameters:
filters: 4
pool_size: 6
num_heads: 2
learning_rate: 0.004179706358511756
head_size_0: 16
Score: 0.7623188495635986

Trial 3 summary
Hyperparameters:
filters: 2
pool_size: 10
num_heads: 2
learning_rate: 0

In [53]:
input_shape = (48000,1)
num_classes = 2
num_blocks = 2

num_of_conv_layers = 1
tuner_2_block_transformer = RandomSearch(
    TransformerHyperModel(input_shape, num_classes, num_blocks),
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='../hpo_tuner/tiny_transformer',
    project_name='tiny_transformer_time_series_2_block_tuning'
)
tuner_2_block_transformer.search(train_dataset, epochs=3, validation_data=val_dataset)
tuner_2_block_transformer.results_summary()

Trial 5 Complete [00h 10m 07s]
val_accuracy: 0.8210144639015198

Best val_accuracy So Far: 0.843478262424469
Total elapsed time: 01h 02m 03s
Results summary
Results in ../hpo_tuner/tiny_transformer/tiny_transformer_time_series_2_block_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 3 summary
Hyperparameters:
filters: 12
pool_size: 8
num_heads: 2
learning_rate: 0.0012440718000666692
head_size_0: 8
head_size_1: 8
Score: 0.843478262424469

Trial 4 summary
Hyperparameters:
filters: 4
pool_size: 10
num_heads: 3
learning_rate: 0.00019658478253735642
head_size_0: 16
head_size_1: 8
Score: 0.8210144639015198

Trial 1 summary
Hyperparameters:
filters: 12
pool_size: 10
num_heads: 1
learning_rate: 0.001413751920126406
head_size_0: 4
head_size_1: 16
Score: 0.7978261113166809

Trial 2 summary
Hyperparameters:
filters: 10
pool_size: 8
num_heads: 3
learning_rate: 0.002291233691844905
head_size_0: 16
head_size_1: 8
Score: 0.75

Trial 0 summary
Hyperparameters:
filte

### Model 2

In [ ]:
best_hps = tuner_2_block_transformer.get_best_hyperparameters(num_trials=1)[0]

print("Best Hyperparameters:")
for param in best_hps.values:
    print(f"{param}: {best_hps.get(param)}")


# filters: 12
# pool_size: 8
# num_heads: 2
# learning_rate: 0.0012440718000666692
# head_size_0: 8
# head_size_1: 8

In [6]:
from tensorflow import keras

In [7]:
def transformer_encoder(inputs, head_size, num_heads, filters, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=filters, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

def build_model(input_shape, head_size, num_heads, filters, num_blocks, num_classes, dropout=0):
    inputs = layers.Input(shape=input_shape)
    
    x = layers.Conv1D(filters=filters, kernel_size=1, activation="relu")(inputs)
    x = layers.MaxPooling1D(pool_size=8)(x)
    x = layers.AveragePooling1D(pool_size=8)(x)
    for i in range(num_blocks):
        x = transformer_encoder(x, head_size, num_heads, filters, dropout)

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(dropout)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)

# Parameters
input_shape = (48000,1)
num_classes = 2
num_blocks = 1

head_size = 8 # 16
num_heads = 2
filters = 12
dropout = 0.1

# Build the model
model_2_10_epochs = build_model(input_shape, head_size, num_heads, filters, num_blocks, num_classes, dropout)
model_2_10_epochs.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                          optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
                          metrics=["accuracy"])

# Model summary
model_2_10_epochs.summary()

# EPOCHS = 1
EPOCHS = 10
history = model_2_10_epochs.fit(
    train_dataset,
    epochs=EPOCHS
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 48000, 1)]           0         []                            
                                                                                                  
 conv1d_3 (Conv1D)           (None, 48000, 12)            24        ['input_2[0][0]']             
                                                                                                  
 max_pooling1d_1 (MaxPoolin  (None, 6000, 12)             0         ['conv1d_3[0][0]']            
 g1D)                                                                                             
                                                                                                  
 average_pooling1d_1 (Avera  (None, 750, 12)              0         ['max_pooling1d_1[0][0]'] 

/home/polina/newname/.venv/lib/python3.9/site-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


353/353 [==============================] - 202s 566ms/step - loss: 0.4602 - accuracy: 0.7380
Epoch 2/10
353/353 [==============================] - 194s 547ms/step - loss: 0.4149 - accuracy: 0.7722
Epoch 3/10
353/353 [==============================] - 191s 540ms/step - loss: 0.3975 - accuracy: 0.7828
Epoch 4/10
353/353 [==============================] - 190s 538ms/step - loss: 0.3853 - accuracy: 0.7885
Epoch 5/10
353/353 [==============================] - 190s 537ms/step - loss: 0.3772 - accuracy: 0.7901
Epoch 6/10
353/353 [==============================] - 192s 541ms/step - loss: 0.3748 - accuracy: 0.7934
Epoch 7/10
353/353 [==============================] - 191s 539ms/step - loss: 0.3698 - accuracy: 0.7914
Epoch 8/10
353/353 [==============================] - 191s 539ms/step - loss: 0.3660 - accuracy: 0.7980
Epoch 9/10
353/353 [==============================] - 190s 537ms/step - loss: 0.3634 - accuracy: 0.7992
Epoch 10/10
353/353 [==============================] - 194s 548ms/step - lo

In [9]:
print("Validation dataset accuracy:")
val_loss, val_acc = model_2_10_epochs.evaluate(x_val_np, y_val_np)

print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
) = predict_and_print_full_results(model_2_10_epochs, x_val_np, y_val_np, model_format="keras")

Validation dataset accuracy:


/home/polina/newname/.venv/lib/python3.9/site-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


44/44 [==============================] - 11s 247ms/step - loss: 0.3446 - accuracy: 0.8065
Validation dataset:
44/44 [==============================] - 11s 245ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 80.65%
Recall: 57.84%
Precision: 80.06%
F1-score: 67.16%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score


KeyboardInterrupt: 

In [ ]:
MODEL_FILE_NAME = "../time_series_models_from_notebooks/tiny_transformer/hpo/tiny_transformer_time_ser_2_block.keras"
model.save(MODEL_FILE_NAME)
print("Model file name: ", MODEL_FILE_NAME)
convert_bytes(get_file_size(MODEL_FILE_NAME), "KB")

### Final model

In [75]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(rate=0.25)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

def build_model(input_shape, head_size, num_heads, ff_dim, num_blocks, num_classes, dropout=0):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv1D(filters=ff_dim, kernel_size=5, activation="relu")(inputs)
    x = layers.MaxPooling1D(pool_size=16)(x)
    x = layers.Dropout(rate=0.25)(x)
    x = layers.AveragePooling1D(pool_size=8)(x)
    for _ in range(num_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(0)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    return models.Model(inputs, outputs)

# Parameters
input_shape = (48000,1)
head_size = 32 
num_heads = 1 # 2
ff_dim = 32 # 32
num_blocks = 1
num_classes = 2
dropout = 0.1

# Build the model
model_lr = build_model(input_shape, head_size, num_heads, ff_dim, num_blocks, num_classes, dropout)
model_lr.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), metrics=["accuracy"])

# Model summary
model_lr.summary()

Model: "model_29"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_31 (InputLayer)       [(None, 48000, 1)]           0         []                            
                                                                                                  
 conv1d_88 (Conv1D)          (None, 47996, 32)            192       ['input_31[0][0]']            
                                                                                                  
 max_pooling1d_30 (MaxPooli  (None, 2999, 32)             0         ['conv1d_88[0][0]']           
 ng1D)                                                                                            
                                                                                                  
 dropout_116 (Dropout)       (None, 2999, 32)             0         ['max_pooling1d_30[0][0

In [76]:
EPOCHS = 10
model_lr.fit(train_dataset, epochs=EPOCHS, validation_data=val_dataset)

print("Validation dataset accuracy:")
val_loss, val_acc = model_lr.evaluate(x_val_np, y_val_np)

Epoch 1/10


353/353 [==============================] - 115s 318ms/step - loss: 0.3661 - accuracy: 0.8467 - val_loss: 0.2215 - val_accuracy: 0.9268
Epoch 2/10
353/353 [==============================] - 115s 323ms/step - loss: 0.2254 - accuracy: 0.9175 - val_loss: 0.1630 - val_accuracy: 0.9449
Epoch 3/10
353/353 [==============================] - 117s 329ms/step - loss: 0.1908 - accuracy: 0.9276 - val_loss: 0.1380 - val_accuracy: 0.9522
Epoch 4/10
353/353 [==============================] - 116s 327ms/step - loss: 0.1741 - accuracy: 0.9335 - val_loss: 0.1301 - val_accuracy: 0.9580
Epoch 5/10
353/353 [==============================] - 116s 326ms/step - loss: 0.1650 - accuracy: 0.9380 - val_loss: 0.1251 - val_accuracy: 0.9609
Epoch 6/10
353/353 [==============================] - 120s 337ms/step - loss: 0.1573 - accuracy: 0.9414 - val_loss: 0.1198 - val_accuracy: 0.9623
Epoch 7/10
353/353 [==============================] - 115s 325ms/step - loss: 0.1512 - accuracy: 0.9426 - val_loss: 0.1103 - val_accura

In [77]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
) = predict_and_print_full_results(model_lr, x_val_np, y_val_np, model_format="keras")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(model_lr, x_test_np, y_test_np, model_format="keras")

print("Time of one prediction for Test dataset:")
evaluate_time_of_prediction(model_lr, x_test_np, y_test_np, model_format="keras", show_prediction_evaluation=True)


Validation dataset:
44/44 [==============================] - 4s 94ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.52%
Recall: 95.34%
Precision: 94.54%
F1-score: 94.94%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9492733997046917

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9490214543072182

Test dataset:
44/44 [==============================] - 4s 101ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.27%
Recall: 94.98%
Precision: 93.75%
F1-score: 94.36%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.942986273519618

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9462869341505324
Time of one prediction for Test dataset:
Accuracy: 96.27%
Recall: 94.98%


In [4]:
MODEL_FILE_NAME = "../time_series_models_from_notebooks/tiny_transformer/hpo/tiny_transformer_time_ser_final.keras"
# model_lr.save(MODEL_FILE_NAME)
print("Model file name: ", MODEL_FILE_NAME)
convert_bytes(get_file_size(MODEL_FILE_NAME), "KB")

Model file name:  ../time_series_models_from_notebooks/tiny_transformer/hpo/tiny_transformer_time_ser_final.keras
File size: 162.422 Kilobytes


In [5]:
model_loaded = tf.keras.models.load_model(MODEL_FILE_NAME)

2024-07-28 17:38:15.865066: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-28 17:38:15.865638: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [8]:
val_loss, val_acc = model_loaded.evaluate(x_val_np, y_val_np)

/home/polina/newname/.venv/lib/python3.9/site-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


44/44 [==============================] - 4s 90ms/step - loss: 0.1031 - accuracy: 0.9652


Tested filters = 8, 16, 32

num_blocks = 1, 2

num_heads = 1, 2

head_size = 8,16, 32

Current implementation is the best